In [ ]:
#An algorithm to derive the DBH (or DAB) out of the point cloud of an individual tree.  Depending on the quality of the
#point cloud, the thresholds can be changed!

In [ ]:
import pandas as pd
import math
import os
import numpy as np
from argparse import Namespace
import pathlib
import matplotlib as plt
import matplotlib.pyplot as mplt
#from shapely import geometry
from mpl_toolkits.mplot3d import Axes3D
import scipy.io as sio
import matplotlib.cm as cmx
import itertools
import scipy.stats as st


In [ ]:
from scipy import optimize
method_2 = "leastsq"

def calc_R(xc, yc):
    """ calculate the distance of each 2D points from the center (xc, yc) """
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def f_2(c):
    """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
    Ri = calc_R(*c)
    return Ri - Ri.mean()

In [ ]:
%matplotlib notebook

In [ ]:
dataDir = ()

In [ ]:
PC = []
for file in os.listdir(dataDir): #or dataDir
    data = pd.read_csv(dataDir + file, header = None, sep = ' ')
    PC.append(data)


In [ ]:
##al tevreden mee
BUT = pd.DataFrame(columns=['xc_1', 'xc_2', 'yc_1', 'yc_2', 'RBH1','RBH2','RES1','RES2','idx','1.4'])
nr = -1
YES = []
NO = []
BAD = [] #niet enough points for a cirlce at 1.3m
NP = [] #not possible, tree too low
DBH1 = []
DBH2 = []
ENKEL13 =[]
for tree in PC:
    nr = nr+1
    sh = min(tree.loc[:,2]) #sh = start height
    eh = max(tree.loc[:,2])#end height
    m1 = 1.27 # 1.27-0.01
    M1 = 1.33 #1.33-0.01
    m2 = 2.27 # 1.27-0.01
    M2 = 2.33
    p = 0
    if eh < sh + M2:
        if eh < sh+ M1:
            NP.append(nr)
        else:
            ENKEL13.append(nr)
        
    else: 
        co_ = tree.loc[m1 + sh <= tree.loc[:,2],:]
        co1 = co_.loc[M1+ sh >= co_.loc[:,2],:]
        co_ = tree.loc[m2 + sh <= tree.loc[:,2],:]
        co2 = co_.loc[M2+ sh >= co_.loc[:,2],:]
    
        x = co1.loc[:,0]
        x2 = co2.loc[:,0]
        y = co1.loc[:,1]
        y2 = co2.loc[:,1]
    
        if len(x)<5 or len(x2)<5: #SUPPOSE 5 POINTS NEEDED TO FIT A CIRCLE
                BAD.append(nr)
        else : 
            x_m1 = np.mean(x)
            y_m1 = np.mean(y)
            x_m2 = np.mean(x2)
            y_m2 = np.mean(y2)

            center_estimate1 = x_m1, y_m1
            center_estimate2 = x_m2, y_m2
            center_1, ier = optimize.leastsq(f_2, center_estimate1)
            x=x2
            y=y2
            center_2, ier = optimize.leastsq(f_2, center_estimate2)
            xc_1_, yc_1_ = center_1
            xc_2_, yc_2_ = center_2
            Ri_1 = calc_R(*center_1)
            Ri_2 = calc_R(*center_2)
    
            R_1 = Ri_1.mean()
            R_2 = Ri_2.mean()
            res1 = np.mean(abs(Ri_1 - R_1)) #sum((Ri_1 - R_1)**2)
            res2 = np.mean(abs(Ri_2 - R_2)) #sum((Ri_2 - R_2)**2)
            RBH1 = R_1
            RBH2 = R_2
            if res1 > 0.03: #.3: #very irregular tree
                m1 = 1.37 
                M1 = 1.43 #try again 1O cm higher
                co_ = tree.loc[m1 + sh <= tree.loc[:,2],:]
                co1 = co_.loc[M1+ sh >= co_.loc[:,2],:]
                y = co1.loc[:,1]
                x = co1.loc[:,0]
                if len(x)<5:
                    BAD.append(nr)
                else:
                    p=1
                    x_m1 = np.mean(x)
                    y_m1 = np.mean(y)
                    center_estimate1 = x_m1, y_m1
                    center_1, ier = optimize.leastsq(f_2, center_estimate1)
                    xc_1_, yc_1_ = center_1
                    Ri_1 = calc_R(*center_1)
                    R_1 = Ri_1.mean()
                    res1 = np.mean(abs(Ri_1 - R_1)) # sum((Ri_1 - R_1)**2)
                    RBH1 = R_1
                    if res1 >= 0.03:  #BUTTRESS
                        YES.append(nr)
                        
            
            ##BUTRESS YES OR NO?
            
            if R_2/R_1>0.82:
                if res1 < 0.03:
                    NO.append(nr)
            else:
                if res1 < 0.03:
                    YES.append(nr)
            df2 = pd.DataFrame({'1.4':[p],'xc_1' : [xc_1_], 'yc_1' : [yc_1_], 'xc_2' : [xc_2_], 'yc_2' : [yc_2_], 'RBH1' : [RBH1],'RBH2' : [RBH2],'RES1' : [res1],'RES2' : [res2],'idx': [nr]})
            BUT = BUT.append(df2, sort = False) 
     


In [ ]:
### script for trees with no buttresses: 5 diameters at 1.3 (or 1.4) meter. 
WO_NB = pd.DataFrame({})
for i in NO:
    tree = PC[i]
    sh = min(tree.loc[:,2]) 
    if BUT.loc[BUT.loc[:,'idx']==i,'1.4'][0]== 0:
        m = 1.27
        M = 1.33 
        z=0
    else:
        m = 1.37
        M = 1.43
        z=1
    co1 = tree.loc[m-0.12 + sh <= tree.loc[:,2],:]
    co_1 = co1.loc[M-0.12 + sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m-0.6 + sh <= tree.loc[:,2],:]
    co_2 = co1.loc[M-0.6 + sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m + sh <= tree.loc[:,2],:]
    co_3 = co1.loc[M+ sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m+0.6 + sh <= tree.loc[:,2],:]
    co_4 = co1.loc[M+0.6 +sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m+0.12 + sh <= tree.loc[:,2],:]
    co_5 = co1.loc[M+0.12 + sh >= co1.loc[:,2],:]
    co = [co_1, co_2, co_3, co_4, co_5]
    TAB= []

    for k in range(len(co)):
        x = co[k].loc[:,0]
        y = co[k].loc[:,1]
        if len(x)<5: #SUPPOSE 5 POINTS NEEDED TO FIT A CIRCLE
            pass
        else:
            x_m = np.mean(x)
            y_m = np.mean(y)
            center_estimate = x_m, y_m
            center_2, ier = optimize.leastsq(f_2, center_estimate)
            xc_2, yc_2 = center_2
            Ri_2 = calc_R(*center_2)
            R_2 = Ri_2.mean()
            res = np.mean(abs(Ri_2 - R_2)) #sum((Ri_2 - R_2)**2)
            TAB.append([res,R_2])
            if k == 3:
                xc_2_ = xc_2
                yx_2_ = yc_2
    b=[]
    c=[]
    for p in TAB:
        if p[0]<0.02: #polishing
            b.append(p[1])
            c.append(p[0])
    if not b:
        for p in TAB:
            if p[0]<0.03: #
                b.append(p[1])
                c.append(p[0])
            
    RBH = np.mean(b)
    RES = np.mean(c)
    AANTAL = len(b)
            
    H_dbh = m+0.03
    df2 = pd.DataFrame({'xc_2' : [xc_2_], 'yc_2' : [yc_2_], 'RBH' : [RBH],'RES' : [RES],'H_dbh': [H_dbh],
                        'AMOUNT':[AANTAL],'idx': [i],'1.4':[z]})
    WO_NB = WO_NB.append(df2, sort = False) 
    
    

In [ ]:
## script for trees with buttresses: using a threshold (minimal error), determine the height at which the diameter should be taken. 
#Take 5 intervals around this height.
WO_B = pd.DataFrame({})
for i in YES:
    tree = PC[i]
    sh = min(tree.loc[:,2]) #sh = start height
    m1 = 1.27-0.02 # 1.27-0.01
    M1 = 1.3-0.02 #1.33-0.01
    m2 = 2.27-0.02 # 1.27-0.01
    M2 = 2.33-0.02
    res = 11
    res_oud = 10
    while res > 0.025 and res_oud != res: # From the moment you are at the branches, the ratio R_2/R_2> 0.82 does not work
        res_oud = res
        m1=m1+0.02
        M1=M1+0.02
        
        co_ = tree.loc[m1 + sh <= tree.loc[:,2],:]
        co1 = co_.loc[M1+ sh >= co_.loc[:,2],:]
    
        x = co1.loc[:,0]
        y = co1.loc[:,1]
        if len(x)<5: #SUPPOSE 5 POINTS NEEDED TO FIT A CIRCLE
            pass
        else: 
            x_m1 = np.mean(x)
            y_m1 = np.mean(y)
    
            center_estimate1 = x_m1, y_m1
            center_1, ier = optimize.leastsq(f_2, center_estimate1)
            xc_1_, yc_1_ = center_1
            Ri_1 = calc_R(*center_1)
            R_1 = Ri_1.mean()
            res = np.mean(abs(Ri_1 - R_1)) #np.mean(np.sqrt((Ri_1 - R_1)**2)) #np.mean(abs(Ri_1 - R_1)) #sum((Ri_1 - R_1)**2)
            RBH1 = R_1
        

    co1 = tree.loc[m1-0.12 + sh <= tree.loc[:,2],:]
    co_1 = co1.loc[M1-0.12 + sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m1-0.6 + sh <= tree.loc[:,2],:]
    co_2 = co1.loc[M1-0.6 + sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m1 + sh <= tree.loc[:,2],:]
    co_3 = co1.loc[M1+ sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m1+0.6 + sh <= tree.loc[:,2],:]
    co_4 = co1.loc[M1+0.6 +sh >= co1.loc[:,2],:]
        
    co1 = tree.loc[m1+0.12 + sh <= tree.loc[:,2],:]
    co_5 = co1.loc[M1+0.12 + sh >= co1.loc[:,2],:]
    co = [co_1, co_2, co_3, co_4, co_5]
    TAB= []

    for k in range(len(co)):
        x = co[k].loc[:,0]
        y = co[k].loc[:,1]
        if len(x)<5: #STEL minstens 5 PTN NODIG VOOR DEFTIGE CIRKEL
            pass
        else:
            x_m = np.mean(x)
            y_m = np.mean(y)
            center_estimate = x_m, y_m
            center_2, ier = optimize.leastsq(f_2, center_estimate)
            xc_2, yc_2 = center_2
            Ri_2 = calc_R(*center_2)
            R_2 = Ri_2.mean()
            res =np.mean(abs(Ri_2 - R_2)) 
            TAB.append([res,R_2])
    b=[]
    c=[]
    for p in TAB:
            if p[0]<0.025: 
                b.append(p[1])
                c.append(p[0])
            
    RBH = np.mean(b)
    RES = np.mean(c)
    AANTAL = len(b)
            
            
    H_dbh = m1+0.03
    df2 = pd.DataFrame({'xc_2' : [xc_1_], 'yc_2' : [yc_1_], 'RBH' : [RBH],'RES' : [RES],'H_dbh': [H_dbh],
                        'AMOUNT':[AANTAL],'idx': [i],'1.4':[z]})
    WO_B = WO_B.append(df2, sort = False) 


In [ ]:
WO_B.to_hdf('DBH_butresses.h5','sleutel')
WO_NB.to_hdf('DBH_nobutresses.h5','sleutel')